In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import lmfit

# from bcqt-ctrl repo
# import helper_functions as hf

from pathlib import Path
from time import sleep, time
# from tqdm.auto import tqdm

%load_ext autoreload
%autoreload 2

In [ ]:
# only S21 dBm has factor of 20, otherwise it'd be a factor of 10
# this is because P = V^2/R, so the V^2 gives an extra factor of 2

def s21_dbm_to_lin(dbm_ampl):
    return np.log10(dbm_ampl) * 20  

def s21_lin_to_dbm(lin_ampl):
    return 10**(lin_ampl/20)

def load_all_csvs_in_path(path, key='*', idx=None):
    """AI is creating summary for load_all_csvs_in_path

    Args:
        path ([type]): [description]
        key (str, optional): [description]. Defaults to '*'.
        idx ([type], optional): [description]. Defaults to None.

    Returns:
        [type]: [description]
    """
    data_files = glob.glob(data_dir+chosen_files)
    
    if idx is not None:  # user wants to load a single file from dir
        idx = 0 if len(data_files) == 1 else idx  # set idx to 0 if there's only one file
        file = data_files[idx]
        df = pd.read_csv(file, sep=",", names=['Frequency','dBm','Phase'])
        freq = np.array(df['Frequency'])
        ampl = s21_lin_to_dbm(df['dBm'])
        phase = np.unwrap(np.deg2rad(df['Phase']))
        cmplx = ampl * np.exp(1j*phase)
        
        return freq, cmplx, df
    
    elif idx is None and len(data_files) != 1:  
        freq_dict, cmplx_dict, df_dict = {}, {}, {}
        for filepath in data_files:
            filename = os.path.basename(filepath)
            df = pd.read_csv(filepath, sep=",", names=['Frequency','dBm','Phase'])
            freq = np.array(df['Frequency'])
            ampl = s21_lin_to_dbm(df['dBm'])
            phase = np.unwrap(np.deg2rad(df['Phase']))
            cmplx = ampl * np.exp(1j*phase)
            
            freq_dict[filename] = freq
            cmplx_dict[filename] = cmplx
            df_dict[filename] = df
            
        return freq_dict, cmplx_dict, df_dict
        
    else:
        # idx is None and len(data_files) == 1
        print(f"idx={idx}, len(data_files)={len(data_files)}\n failed if & elif")
        raise
        

# load data from sample directory
data_dir = '../sample_data/R0_Jorge/'
chosen_files = '*'
# display(glob.glob(data_dir+chosen_files))
freq, cmplx, df = load_all_csvs_in_path(data_dir, chosen_files, idx=1)

print(freq, "\n", cmplx)


## Testing scresonators

In [ ]:
cur_dir = os.getcwd()
parent_parent_dir = os.path.dirname(os.path.dirname(cur_dir))

# this notebook is located in E:/GitHub/scresonators/test_ontebooks
# so the parent's parent directory is just E:/GitHub
# and by adding that to our path, we can import E:/GitHub/scresonators
sys.path.append(parent_parent_dir)
print(os.getcwd())

import scresonators.src as scres
print("\nimported scresonators!\n")


def print_dir(module, filter_underscore=True, text=""):
    if filter_underscore == True:
        print("showing dir({})".format(text))
        display( [i for i in dir(module) if '__' not in i] )
    else:
        print("showing dir({})".format(text))
        display( dir(module) )


print_dir(scres.fit_methods.dcm, True, "src.fit_methods.dcm")
print_dir(scres.fit_methods.dcm.DCM, True, "src.fit_methods.dcm.DCM")

dcm_method = scres.fit_methods.dcm.DCM()
FitRes = scres.Fitter(dcm_method)

w1 = freq[np.abs(cmplx).argmin()]
init_guess = {
    'Q' : {'value' : 1e6, 'min' : 1e3, 'max' : 1e9},
    'Qc' : {'value' : 1e5, 'min' : 1e3, 'max' : 1e9},
    'w1' : {'value' : w1, 'min' : w1-3e3, 'max' : w1+3e3, 'vary' : True},
    'phi' : {'value' : 0.08, 'min' : -np.pi/2, 'max' : np.pi/2}, 
}

init_params = lmfit.create_params(**init_guess)
init_params.pretty_print()
print()

amps = np.abs(cmplx)
db_amps = np.log10(amps) * 20

phases = np.angle(cmplx)
result, conf_intervals = FitRes.fit(freq, db_amps, phases, manual_init=init_params, verbose=True)

fit_result_params = result.params


In [ ]:
fit_result_params.pretty_print()

y_fit = result.eval(params=fit_result_params, x=freq)
# y_fit_err = result.eval_uncertainty()
Q_val = fit_result_params["Q"].value
Q_err = fit_result_params["Q"].stderr
dQ = 100 * Q_err/Q_val
print(dQ, Q_val, Q_err)

print("\nabs(data): ", np.abs(cmplx))
print("abs(result): ", np.abs(y_fit))

##########################################
#############    plotting   ##############
##########################################

mosaic = "AAAA\n BBCC"
fig, axes = plt.subplot_mosaic(mosaic, figsize=(7,8))
ax1, ax2, ax3 = axes["A"], axes["B"], axes["C"]

ax1.plot(freq, np.abs(cmplx), label="Data")
ax1.plot(freq, np.abs(y_fit), label="Fit")
ax1.set_yscale("log")
ax1.legend()
ax1.set_title("S21 Magnitude")

ax2.plot(np.real(cmplx), np.imag(cmplx), label="Data", linestyle='', marker='o')
ax2.set_title("Data")

ax3.plot(np.real(y_fit), np.imag(y_fit), label="Fit", linestyle='', marker='o', color='orange')
ax3.set_title("Fit Result")


fig.tight_layout()

## Multiple Resonators

to be implemented

In [ ]:
# # load data from Andre
# data_dir = './samples/R0_Jorge/'
# chosen_files = '*'

# freq_dict, cmplx_dict = load_csv(data_dir, chosen_files, idx=None)

# dQ_list, Q_list = [], []

# print(f"# of resonators: {len(freq_dict)}")

# for freq, cmplx in zip(freq_dict.values(), cmplx_dict.values()):

#     FitRes = scres.Fitter(scres.fit_methods.DCM)

#     w1 = freq[np.abs(cmplx).argmin()]
#     init_guess = {
#         'Q' : {'value' : 1e6, 'min' : 1e3, 'max' : 1e9},
#         'Qc' : {'value' : 1e5, 'min' : 1e3, 'max' : 1e9},
#         'w1' : {'value' : w1, 'min' : w1-3e3, 'max' : w1+3e3, 'vary' : True},
#         'phi' : {'value' : 0.08, 'min' : -np.pi/2, 'max' : np.pi/2}, 
#     }

#     init_params = lmfit.create_params(**init_guess)

#     amps = np.abs(cmplx)
#     db_amps = np.log10(amps) * 20

#     phases = np.angle(cmplx)
#     result, conf_intervals = FitRes.fit(freq, db_amps, phases, manual_init=init_params, verbose=True)

#     fit_params = result.params
        
#     y_fit = result.eval(params=fit_params, x=freq)
#     # y_fit_err = result.eval_uncertainty()
#     Q_val = fit_params["Q"].value
#     Q_err = fit_params["Q"].stderr
#     dQ = 100 * Q_err/Q_val
#     print(dQ, Q_val, Q_err)
    
#     dQ_list.append(dQ)
#     Q_list.append(Q_val)


    

In [ ]:
# for dQ, Q in zip(dQ_list, Q_list):
    
#     print(f"{dQ:1.2f}, {Q:1.2f}")

# fig, axes = plt.subplots(2,1, figsize=(7,8), sharex=True)
# ax1, ax2 = axes[0], axes[1]

# ax1.plot(dQ_list, 'ro', label="dQ")
# ax1.set_title(r"dQ Values  ($dQ=100 * dQ/\sigma dQ$)")

# ax2.plot(Q_list, 'bo', label="Fit")
# ax2.set_title("Q Values")

# ax2.set_xlabel("Resonator #")